<a href="https://colab.research.google.com/github/ravy-dotcom/apiw/blob/main/Crawling_Berita_Detikcom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Aktifkan Packages
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

In [ ]:
#Cek apakah URL dapat diakses
#Ganti link URL yang anda inginkan
#Jika response [200] maka dapat terkoneksi

url = "https://www.detik.com/jabar/berita/d-8065912/perjalanan-kasus-korupsi-setnov-hingga-bebas-bersyarat"
response = requests.get(url)
response

<Response [200]>

Fungsi Crawling Berita di Detikcom

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime
import time
import random

def scraping_artikel_detikcom(url):
    # Memanggil url
    response = requests.get(url)

    # Parsing html
    soup = BeautifulSoup(response.text, 'html.parser')

    # Mengekstrak judul
    judul = re.sub(r'^\s+|\s+$', '', soup.find('h1').text)

    # Mengekstrak penulis
    penulis = re.sub(r'\s*-\s*detikJabar$', '', soup.find('div', class_='detail__author').text)

    # Mengekstrak waktu rilis
    waktu_rilis = soup.find('div', class_='detail__date').text
    pattern = r'(?P<tanggal>\d{2}\s\w{3}\s\d{4})\s+(?P<jam>\d{2}:\d{2})'
    match = re.search(pattern, waktu_rilis)

    bulan_map = {
        "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "Mei": "05",
        "Jun": "06", "Jul": "07", "Agu": "08", "Sep": "09", "Okt": "10",
        "Nov": "11", "Des": "12"
    }

    if match:
        tanggal_str = match.group("tanggal")  # '02 Agu 2025'
        jam_str = match.group("jam")          # '15:30'

        # pecah tanggal
        hari, bulan_txt, tahun = tanggal_str.split()
        bulan_num = bulan_map[bulan_txt]

        # buat string standar
        tanggal_iso = f"{tahun}-{bulan_num}-{hari}"

        # parse ke datetime.date
        tanggal_date = datetime.strptime(tanggal_iso, "%Y-%m-%d").date()

    # Mengekstrak isi berita
    isi = ' '.join([p.text for p in soup.find('div', class_='detail__body-text itp_bodycontent').find_all('p', class_=False)])

    # Menggabungkan hasil ekstraksi
    artikel = {'judul':judul, 'penulis': penulis, 'tanggal': tanggal_date, 'jam': jam_str, 'isi': isi, 'url': url}

    return artikel

Jika hanya mengambil 1 isi artkel tinggal inputkan URLnya seperti dibawah (Jangan lupa running "Fungsi Crawling Berita di Detikcom" terlebih dahulu)

In [ ]:
scraping_artikel_detikcom('https://www.detik.com/jabar/berita/d-8065912/perjalanan-kasus-korupsi-setnov-hingga-bebas-bersyarat')

{'judul': 'Perjalanan Kasus Korupsi Setnov hingga Bebas Bersyarat',
 'penulis': 'Rifat Alhamidi',
 'tanggal': datetime.date(2025, 8, 18),
 'jam': '08:00',
 'isi': 'Mantan Ketua DPR RI Setya Novanto mendapat pembebasan bersyarat. Novanto merupakan terpidana kasus korupsi proyek e-KTP yang divonis hukuman 15 tahun kurungan penjara pada 2018 silam. Namun kemudian, perlawanan masih dilayangkan Setya Novanto melalui peninjauan kembali (PK) ke Mahkamah Agung (MA). Hukumannya pun kemudian disunat menjadi 12 tahun 6 bulan, dan akhirnya Novanto bebas pada Sabtu (16/8/2025) kemarin dari Lapas Sukamiskin Bandung.Lantas, bagaimana perjalanan kasus korupsi Setya Novanto dimulai? Berikut ini rangkuman detikJabar: Setya Novanto dijebloskan ke penjara setelah ditetapkan menjadi tersangka oleh KPK pada September 2017. Novanto saat itu terseret kasus korupsi e-KTP bersama tersangka lain seperti pejabat Kemendagri, Irman dan Sugiharto. Namun kemudian, Setya Novanto bebas setelah praperadilan yang dia lay

Jika ingin mengambil berita dari beberapa URL ikuti langkah dibawah ini (Jangan lupa running "Fungsi Crawling Berita di Detikcom" terlebih dahulu)

In [ ]:
listurls = [
    'https://www.detik.com/bali/hukum-dan-kriminal/d-8054115/rk-dan-lisa-mariana-tes-dna-bagaimana-hasilnya',
    'https://sport.detik.com/sepakbola/liga-indonesia/d-8065429/erick-thohir-tegaskan-tidak-ada-masalah-timnas-pakai-lagu-kebangsaan?utm_source=pbnews&utm_medium=display&utm_campaign=promokonten&utm_term=20detikdhattrickags25erickthohir&utm_content=20detik',
    'https://www.detik.com/jabar/jabar-gaskeun/d-8066550/jadwal-sim-keliling-bandung-terbaru-18-23-agustus-2025-cek-lokasinya',
    'https://news.detik.com/berita/d-8066587/eks-penyidik-kpk-dorong-ada-regulasi-batasi-koruptor-dapat-bebas-bersyarat',
    'https://news.detik.com/berita/d-8066584/sekjen-mpr-ajak-generasi-muda-maknai-perjuangan-pahlawan']

#URL yang sudah kita list akan kita ambil sesuai dengan fungsi yang kita buat sebelumnya ("Fungsi Crawling Berita di Detikcom")
listartikel = []
for url in listurls:
    artikel = scraping_artikel_detikcom(url)
    listartikel.append(artikel)

df_artikel = pd.DataFrame(listartikel)

df_artikel

,judul,penulis,tanggal,jam,isi,url
0,"RK dan Lisa Mariana Tes DNA, Bagaimana Hasilnya?",Tim detikNews - detikBali,2025-08-10,15:46,Ridwan Kamil (RK) akhirnya bertemu di meja pen...,https://www.detik.com/bali/hukum-dan-kriminal/...
1,Erick Thohir Tegaskan Tidak Ada Masalah Timnas...,Afif Farhan - Sepakbola,2025-08-17,14:00,Belakangan ini ramai soal klaim komersil lagu ...,https://sport.detik.com/sepakbola/liga-indones...
2,Jadwal SIM Keliling Bandung Terbaru 18-23 Agus...,Wisma Putra,2025-08-18,08:15,Bagi warga Kota Bandung yang ingin memperpanja...,https://www.detik.com/jabar/jabar-gaskeun/d-80...
3,Eks Penyidik KPK Dorong Ada Regulasi Batasi Ko...,Farih Maulana Sidik - detikNews,2025-08-18,08:45,Mantan penyidik KPK Yudi Purnomo Harahap menyo...,https://news.detik.com/berita/d-8066587/eks-pe...
4,Sekjen MPR Ajak Generasi Muda Maknai Perjuanga...,Dea Duta Aulia - detikNews,2025-08-18,08:36,Sekretaris Jenderal (Sekjen) MPR RI Siti Fauzi...,https://news.detik.com/berita/d-8066584/sekjen...


Jika ingin mengambil berita dari 1 atau bahkan lebih dari 1 halaman ikuti langkah dibawah ini (Jangan lupa running "Fungsi Crawling Berita di Detikcom" terlebih dahulu)

In [ ]:
#mengambil hanya berita bukan gambar, video, dll
def urls_perpage_detikcom(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    urls = [i.find('a').get('href') for i in soup.find_all('article')]
    filtered_urls = [u for u in urls if u.startswith("https://www.detik.com/jabar/berita")]
    return filtered_urls

#mengambil berita sesuai banyak halaman yang kita piih
#rabge (1,3) artinya mengambil berita halaman 1, 2 dan 3
listurls = []
for i in list(range(1, 3)):
    url = 'https://www.detik.com/tag/ciamis/?sortby=time&page='+str(i)
    listurls.append(urls_perpage_detikcom(url))
    time.sleep(random.uniform(1,2))
flat_urls = [url for sublist in listurls for url in sublist]
flat_urls

['https://www.detik.com/jabar/berita/d-8079932/perjuangan-arif-cari-kerja-lewat-job-fair-di-ciamis-usai-tergeser-ai',
 'https://www.detik.com/jabar/berita/d-8078031/gerombolan-monyet-liar-rusak-sawah-dan-kebun-warga-di-ciamis',
 'https://www.detik.com/jabar/berita/d-8077486/dini-hari-mencekam-saat-rumah-lansia-di-ciamis-dilalap-api',
 'https://www.detik.com/jabar/berita/d-8073449/king-kobra-3-meter-teror-warga-sukamaju-ciamis',
 'https://www.detik.com/jabar/berita/d-8072514/rumah-lansia-90-tahun-di-cipaku-ciamis-ludes-terbakar',
 'https://www.detik.com/jabar/berita/d-8072271/hilang-10-hari-pamit-cari-ikan-jepri-ditemukan-tewas-di-sungai-citanduy',
 'https://www.detik.com/jabar/berita/d-8065109/mengenal-dua-sosok-pahlawan-nasional-asal-ciamis',
 'https://www.detik.com/jabar/berita/d-8064378/priangan-sepekan-gelombang-penolakan-keramba-jaring-apung',
 'https://www.detik.com/jabar/berita/d-8064038/haru-pecah-di-pendopo-ciamis-rasa-bangga-ortu-kala-anak-jadi-paskibra',
 'https://www.detik.

In [ ]:
#URL yang sudah terlist dari beberapa halaman akan kita ambil sesuai dengan fungsi yang kita buat sebelumnya ("Fungsi Crawling Berita di Detikcom") Jika ada URL yang tidak tercrawling maka akan terhitung masuk ke gagal
listartikel = []
gagal = []

for url in flat_urls:
    try:
        artikel = scraping_artikel_detikcom(url)
        listartikel.append(artikel)
    except Exception as e:
        gagal.append({"url": url, "error": str(e)})
    time.sleep(random.uniform(1, 2))

print("Jumlah berhasil:", len(listartikel))
print("Jumlah gagal:", len(gagal))

Jumlah berhasil: 14
Jumlah gagal: 0


In [ ]:
#Jadikan berita yang sudah di crawling sebagai data frame
df=pd.DataFrame(listartikel)
df

,judul,penulis,tanggal,jam,isi,url
0,Perjuangan Arif Cari Kerja Lewat Job Fair di C...,Dadang Hermansyah,2025-08-27,06:00,Suasana Polres Ciamis Selasa (26/8/2025) nampa...,https://www.detik.com/jabar/berita/d-8079932/p...
1,Gerombolan Monyet Liar Rusak Sawah dan Kebun W...,Dadang Hermansyah,2025-08-25,19:29,Hijaunya hamparan sawah dan rindangnya pepohon...,https://www.detik.com/jabar/berita/d-8078031/g...
2,Dini Hari Mencekam Saat Rumah Lansia di Ciamis...,Dadang Hermansyah,2025-08-25,09:50,"Warga Dusun Bantarsari, Desa Awiluar, Kecamata...",https://www.detik.com/jabar/berita/d-8077486/d...
3,King Kobra 3 Meter Teror Warga Sukamaju Ciamis,Dadang Hermansyah,2025-08-22,09:50,Teror sang 'raja ular' alias king kobra mengha...,https://www.detik.com/jabar/berita/d-8073449/k...
4,Rumah Lansia 90 Tahun di Cipaku Ciamis Ludes T...,Dadang Hermansyah,2025-08-21,16:45,"Suasana sepi di Dusun Sindangjaya, Desa Ciakar...",https://www.detik.com/jabar/berita/d-8072514/r...
5,"Hilang 10 Hari Pamit Cari Ikan, Jepri Ditemuka...",Dadang Hermansyah,2025-08-21,15:15,Suasana tenang Kamis (21/8/2025) siang di Ling...,https://www.detik.com/jabar/berita/d-8072271/h...
6,Mengenal Dua Sosok Pahlawan Nasional Asal Ciamis,Dadang Hermansyah,2025-08-17,17:30,Di tengah momen peringatan Hari Kemerdekaan In...,https://www.detik.com/jabar/berita/d-8065109/m...
7,Priangan Sepekan: Gelombang Penolakan Keramba ...,Tim detikJabar,2025-08-17,12:00,Sepekan ini berbagai peristiwa terjadi di wila...,https://www.detik.com/jabar/berita/d-8064378/p...
8,"Haru Pecah di Pendopo Ciamis, Rasa Bangga Ortu...",Dadang Hermansyah,2025-08-16,17:00,Air mata bahagia pecah di Halaman Pendopo Bupa...,https://www.detik.com/jabar/berita/d-8064038/h...
9,"Bukan Naikkan Tarif PBB, Ini Cara Ciamis Tingk...",Dadang Hermansyah,2025-08-15,15:00,Saat sejumlah daerah menaikkan tarif Pajak Bum...,https://www.detik.com/jabar/berita/d-8062471/b...


In [ ]:
#simpan dalam format csv
df.to_csv('dataDetikcom3Halaman.csv', index=False)

In [ ]:
#simpan dalam format excel
df.to_excel('dataDetikcom3Halaman.xlsx', index=False)